In [2]:

import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  # accessor needed to parse crs
import calendar
import argparse
import seaborn as sns
from shapely.geometry import Point
import geopandas
from geodatasets import get_path
import glob
import os

In [4]:
input_dir_path = '/group/moniergrp/TEM_Large_Ensemble/input_files/large_ensemble/' 
output_dir_path = '/group/moniergrp/TEM_Large_Ensemble/output_files/' 
region = 'region_1/'

In [8]:
clm_files = pd.DataFrame({'path':os.listdir(input_dir_path+region+'climate/future/')})
clm_files[['model', 'scenario', 'variable']] = clm_files['path'].str.split('_',expand=True)

In [18]:
clm_files['model_scenario'] = clm_files['model'] + clm_files['scenario']

In [19]:
clm_files['model_scenario'].unique()

array(['MPIxxESM1xx2xxHRssp585', 'MRIxxESM2xx0ssp460',
       'MPIxxESM1xx2xxHRssp126', 'BCCxxCSM2xxMRssp370',
       'AWIxxCMxx1xx1xxMRssp126', 'MPIxxESM1xx2xxLRssp370',
       'CanESM5ssp460', 'MPIxxESM1xx2xxLRssp245', 'MRIxxESM2xx0ssp245',
       'MIROC6ssp434', 'CanESM5ssp245', 'MIROC6ssp460',
       'AWIxxCMxx1xx1xxMRssp585', 'MRIxxESM2xx0ssp434',
       'MPIxxESM1xx2xxHRssp245', 'CanESM5ssp585',
       'AWIxxCMxx1xx1xxMRssp245', 'MPIxxESM1xx2xxLRssp126',
       'AWIxxCMxx1xx1xxMRssp370', 'MPIxxESM1xx2xxHRssp370',
       'MPIxxESM1xx2xxLRssp585', 'CanESM5ssp434', 'MIROC6ssp245',
       'BCCxxCSM2xxMRssp126', 'BCCxxCSM2xxMRssp585', 'MIROC6ssp370',
       'MRIxxESM2xx0ssp370', 'MIROC6ssp119', 'MRIxxESM2xx0ssp119',
       'MIROC6ssp585', 'BCCxxCSM2xxMRssp245', 'MRIxxESM2xx0ssp585',
       'MRIxxESM2xx0ssp126', 'MIROC6ssp126'], dtype=object)

In [13]:
input_col_names = ['lon', 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [14]:
i=0
clm = pd.read_csv(input_dir_path+region+'climate/future/'+clm_files.loc[i,'path'], names = input_col_names )
# clm
var = clm.loc[0,'var'].replace(" ", "")
clm = clm[['lon', 'lat', 'Area', 'year', 'average', 'min', 'max']]
clm=clm.rename(columns={'average':var+'_avg'
                        ,'min':var+'_min'
                        ,'max':var+'_max'})

In [15]:
clm

,lon,lat,Area,year,tair_avg,tair_min,tair_max
0,-177.0,-44.0,991,2015,14.08,11.35,16.73
1,-177.0,-44.0,991,2016,14.21,10.86,18.55
2,-177.0,-44.0,991,2017,14.09,11.07,17.66
3,-177.0,-44.0,991,2018,13.94,11.23,17.45
4,-177.0,-44.0,991,2019,13.49,11.15,17.17
...,...,...,...,...,...,...,...
55293,-120.0,54.0,1806,2096,7.02,-8.48,24.29
55294,-120.0,54.0,1806,2097,7.45,-5.29,21.70
55295,-120.0,54.0,1806,2098,7.44,-3.74,21.88
55296,-120.0,54.0,1806,2099,7.68,-6.01,21.38


In [23]:
clm_merge.columns

Index(['lon', 'lat', 'Area', 'year', 'tair_avg', 'tair_min', 'tair_max',
       'trange_avg', 'trange_min', 'trange_max', 'wind_avg', 'wind_min',
       'wind_max', 'nirr_avg', 'nirr_min', 'nirr_max', 'vpr_avg', 'vpr_min',
       'vpr_max', 'prec_avg', 'prec_min', 'prec_max', 'model', 'scenario'],
      dtype='object')

In [26]:
all_model_scenarios = pd.DataFrame(columns = ['model', 'scenario','lon', 'lat', 'Area', 'year', 'tair_avg', 'tair_min', 'tair_max',
       'trange_avg', 'trange_min', 'trange_max', 'wind_avg', 'wind_min',
       'wind_max', 'nirr_avg', 'nirr_min', 'nirr_max', 'vpr_avg', 'vpr_min',
       'vpr_max', 'prec_avg', 'prec_min', 'prec_max'])
model_scenarios = clm_files['model_scenario'].unique()

for m_s in model_scenarios:
    loop_clm_files = clm_files.loc[clm_files['model_scenario'] == m_s].reset_index()
    for i in range(len(loop_clm_files)):
        clm = pd.read_csv(input_dir_path+region+'climate/future/'+loop_clm_files.loc[i,'path'], names = input_col_names )
        var = clm.loc[0,'var'].replace(" ", "")
        clm = clm[['lon', 'lat', 'Area', 'year', 'average', 'min', 'max']]
        clm=clm.rename(columns={'average':var+'_avg'
                                ,'min':var+'_min'
                                ,'max':var+'_max'})

        if (i==0):
            clm_merge = clm
        else:
            clm_merge = clm_merge.merge(clm, on = ['lon', 'lat', 'Area', 'year'])
        # print(len(clm_merge))
        # print(clm_merge.columns)
    clm_merge['model'] = loop_clm_files.loc[0, 'model']
    clm_merge['scenario'] = loop_clm_files.loc[0, 'scenario']
    all_model_scenarios = pd.concat([all_model_scenarios, clm_merge])
    print(str(len(loop_clm_files)), m_s)
all_model_scenarios.to_csv(
    output_dir_path+region+'future_climate_inputs.csv'
,index=False)

6 MPIxxESM1xx2xxHRssp585
6 MRIxxESM2xx0ssp460
6 MPIxxESM1xx2xxHRssp126
6 BCCxxCSM2xxMRssp370
6 AWIxxCMxx1xx1xxMRssp126
6 MPIxxESM1xx2xxLRssp370
6 CanESM5ssp460
6 MPIxxESM1xx2xxLRssp245
6 MRIxxESM2xx0ssp245
6 MIROC6ssp434
6 CanESM5ssp245
6 MIROC6ssp460
6 AWIxxCMxx1xx1xxMRssp585
6 MRIxxESM2xx0ssp434
6 MPIxxESM1xx2xxHRssp245
6 CanESM5ssp585
6 AWIxxCMxx1xx1xxMRssp245
6 MPIxxESM1xx2xxLRssp126
6 AWIxxCMxx1xx1xxMRssp370
6 MPIxxESM1xx2xxHRssp370
6 MPIxxESM1xx2xxLRssp585
6 CanESM5ssp434
6 MIROC6ssp245
6 BCCxxCSM2xxMRssp126
6 BCCxxCSM2xxMRssp585
6 MIROC6ssp370
6 MRIxxESM2xx0ssp370
6 MIROC6ssp119
6 MRIxxESM2xx0ssp119
6 MIROC6ssp585
6 BCCxxCSM2xxMRssp245
6 MRIxxESM2xx0ssp585
6 MRIxxESM2xx0ssp126
6 MIROC6ssp126


In [27]:
all_model_scenarios.to_csv(
    output_dir_path+region+'future_climate_inputs.csv'
,index=False)